In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
from splinter import Browser
import time 

In [2]:
executable_path = {'executable_path' : ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\10pai\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
csv_path = '2021RidersV2.csv'
riders_v2 = pd.read_csv(csv_path, names = ['Year', 'League', 'Bike Number', 'Name', 'Nationality', 'Team', 'Manufacturer'])
riders_v2

Year  League  Bike Number               Name Nationality  \
0    2021  MotoGP           21  Franco MORBIDELLI         ITA   
1    2021  MotoGP           41    Aleix ESPARGARO         SPA   
2    2021  MotoGP           43        Jack MILLER         AUS   
3    2021  MotoGP           20   Fabio QUARTARARO         FRA   
4    2021  MotoGP           36           Joan MIR         SPA   
..    ...     ...          ...                ...         ...   
117  2021   MotoE            6      Maria HERRERA         SPA   
118  2021   MotoE           19  Corentin PEROLARI         FRA   
119  2021   MotoE           80       Jasper IWEMA         NED   
120  2021   MotoE           21      Kevin ZANNONI         ITA   
121  2021   MotoE           14        Andre PIRES         POR   

                             Team Manufacturer  
0             Petronas Yamaha SRT       Yamaha  
1     Aprilia Racing Team Gresini      Aprilia  
2              Ducati Lenovo Team       Ducati  
3    Monster Energy Yamaha MotoGP       Yamaha  
4              Team SUZUKI ECSTAR       Suzuki  
..                            ...          ...  
117           OpenBank Aspar Team     Energica  
118               Tech 3 E-Racing     Energica  
119                Pons Racing 40     Energica  
120                    LCR E-Team     Energica  
121    Avintia Esponsorama Racing     Energica  

[122 rows x 7 columns]

In [4]:
list_of_riders = []        
    
for pd_name in riders_v2['Name']:
    riders_stats = {}
    name = pd_name.title().replace(" ", "+", 1)
    try:
        url = "https://www.motogp.com/en/riders/profile/" + name
        browser.visit(url)

        time.sleep(7)

        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        riders_stats['name'] = pd_name
        birthdate = soup.find('p', class_ = 'c-rider-birth_date').text
        riders_stats['birthdate'] = birthdate[17 : -1]

        stats = soup.find('p', class_ = 'c-rider-attributes').text
        nstats = stats.split(' | ')
        riders_stats['height'] = nstats[0].replace(' Height : ', '')
        riders_stats['weight'] = nstats[1].replace('Weight : ', '')



    except:
        riders_stats['name'] = pd_name
        riders_stats['birthdate'] = 'Vampyre'
        riders_stats['height'] = 'Andre the Giant'
        riders_stats['weight'] = 'Atom'


    list_of_riders.append(riders_stats)
    
browser.quit()
        

In [5]:
list_of_riders

[{'name': 'Franco MORBIDELLI',
  'birthdate': '1994-12-04',
  'height': '176 cm',
  'weight': '64 kg '},
 {'name': 'Aleix ESPARGARO',
  'birthdate': '1989-07-30',
  'height': '180 cm',
  'weight': '66 kg '},
 {'name': 'Jack MILLER',
  'birthdate': '1995-01-18',
  'height': '173 cm',
  'weight': '64 kg '},
 {'name': 'Fabio QUARTARARO',
  'birthdate': '1999-04-20',
  'height': '177 cm',
  'weight': '66 kg '},
 {'name': 'Joan MIR',
  'birthdate': '1997-09-01',
  'height': '181 cm',
  'weight': '69 kg '},
 {'name': 'Maverick VIÑALES',
  'birthdate': '1995-01-12',
  'height': '171 cm',
  'weight': '64 kg '},
 {'name': 'Alex RINS',
  'birthdate': '1995-12-08',
  'height': '176 cm',
  'weight': '72 kg '},
 {'name': 'Francesco BAGNAIA',
  'birthdate': '1997-01-14',
  'height': '176 cm',
  'weight': '67 kg '},
 {'name': 'Valentino ROSSI',
  'birthdate': '1979-02-16',
  'height': '181 cm',
  'weight': '69 kg '},
 {'name': 'Miguel OLIVEIRA',
  'birthdate': '1995-01-04',
  'height': '170 cm',
  'w

In [6]:
riders_df = pd.DataFrame(list_of_riders)
riders_df['name'].nunique()
riders_df.drop_duplicates(inplace = True)
riders_df.reset_index(drop = True, inplace = True)
riders_df.to_csv('birfdays.csv')


In [7]:
riders_df

name   birthdate  height  weight
0    Franco MORBIDELLI  1994-12-04  176 cm  64 kg 
1      Aleix ESPARGARO  1989-07-30  180 cm  66 kg 
2          Jack MILLER  1995-01-18  173 cm  64 kg 
3     Fabio QUARTARARO  1999-04-20  177 cm  66 kg 
4             Joan MIR  1997-09-01  181 cm  69 kg 
..                 ...         ...     ...     ...
98       Maria HERRERA  1996-08-26  162 cm  54 kg 
99   Corentin PEROLARI  1998-04-13  170 cm  65 kg 
100       Jasper IWEMA  1989-11-15  169 cm  69 kg 
101      Kevin ZANNONI  2000-09-08  172 cm  63 kg 
102        Andre PIRES  1989-08-29  161 cm  61 kg 

[103 rows x 4 columns]